In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from os import listdir
from os.path import isfile, join
import numpy as np
import tensorflow as tf
from PIL import Image as img
from tensorflow.python import keras
from keras import Sequential, datasets, layers, models
from keras.preprocessing.image import ImageDataGenerator
import pickle as pk

In [2]:
pixels = 150
dimensions = (pixels, pixels)
bins = 2
mid_layer = 4096
num_epochs = 3
learning_rate = 0.001
reduction_factor = 0.8

In [3]:
class LearningRateReducerCb(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        old_lr = self.model.optimizer.lr.read_value()
        new_lr = old_lr * reduction_factor
        print("\nEpoch: {}. Reducing Learning Rate from {} to {}".format(epoch, old_lr, new_lr))
        self.model.optimizer.lr.assign(new_lr)

In [4]:
train_images = []
train_labels = np.array([])

folder_path = "/mnt/c/NN/Brain_Tumor/Training/"

for i in range(10, 1321):
    im = img.open(folder_path + "glioma/Tr-gl_" + str(i).zfill(4) + ".jpg")
    im = im.resize(dimensions)
    im = im.convert('L')
    im_array = np.asarray(im)
    im.close()
    train_images.append(im_array)
    train_labels = np.append(train_labels, 1)

for i in range(10, 1339):
    im = img.open(folder_path + "meningioma/Tr-me_" + str(i).zfill(4) + ".jpg")
    im = im.resize(dimensions)
    im = im.convert('L')
    im_array = np.asarray(im)
    im.close()
    train_images.append(im_array)
    train_labels = np.append(train_labels, 1)

for i in range(10, 1457):
    im = img.open(folder_path + "pituitary/Tr-pi_" + str(i).zfill(4) + ".jpg")
    im = im.resize(dimensions)
    im = im.convert('L')
    im_array = np.asarray(im)
    im.close()
    train_images.append(im_array)
    train_labels = np.append(train_labels, 1)

for i in range(10, 1595):
    im = img.open(folder_path + "notumor/Tr-no_" + str(i).zfill(4) + ".jpg")
    im = im.resize(dimensions)
    im = im.convert('L')
    im_array = np.asarray(im)
    im.close()
    train_images.append(im_array)
    train_labels = np.append(train_labels, 0)

In [6]:
test_images = []
test_labels = np.array([])

folder_path = "/mnt/c/NN/Brain_Tumor/Testing/"

for i in range(10, 300):
    im = img.open(folder_path + "glioma/Te-gl_" + str(i).zfill(4) + ".jpg")
    im = im.resize(dimensions)
    im = im.convert('L')
    im_array = np.asarray(im)
    im.close()
    test_images.append(im_array)
    test_labels = np.append(test_labels, 1)

for i in range(10, 306):
    im = img.open(folder_path + "meningioma/Te-me_" + str(i).zfill(4) + ".jpg")
    im = im.resize(dimensions)
    im = im.convert('L')
    im_array = np.asarray(im)
    im.close()
    test_images.append(im_array)
    test_labels = np.append(test_labels, 1)

for i in range(10, 300):
    im = img.open(folder_path + "pituitary/Te-pi_" + str(i).zfill(4) + ".jpg")
    im = im.resize(dimensions)
    im = im.convert('L')
    im_array = np.asarray(im)
    im.close()
    test_images.append(im_array)
    test_labels = np.append(test_labels, 1)

for i in range(10, 405):
    im = img.open(folder_path + "notumor/Te-no_" + str(i).zfill(4) + ".jpg")
    im = im.resize(dimensions)
    im = im.convert('L')
    im_array = np.asarray(im)
    im.close()
    test_images.append(im_array)
    test_labels = np.append(test_labels, 0)

In [7]:
train_images = np.array(train_images)
test_images = np.array(test_images)

In [8]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(pixels, pixels, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding="same"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding="same"))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding="same"))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding="same"))
model.add(layers.MaxPooling2D((3, 3)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(bins, activation='sigmoid'))

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [9]:
history = model.fit(train_images, train_labels, callbacks=[LearningRateReducerCb()], epochs=num_epochs,
                    validation_data=(test_images, test_labels))

Epoch 1/3


/home/mparanja/miniconda3/envs/mlenv/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


178/178 [==============================] - ETA: 0s - loss: 0.6801 - accuracy: 0.8875
Epoch: 0. Reducing Learning Rate from 0.0010000000474974513 to 0.000800000037997961
178/178 [==============================] - 85s 472ms/step - loss: 0.6801 - accuracy: 0.8875 - val_loss: 0.3343 - val_accuracy: 0.9079
Epoch 2/3
178/178 [==============================] - ETA: 0s - loss: 0.1165 - accuracy: 0.9647
Epoch: 1. Reducing Learning Rate from 0.000800000037997961 to 0.000640000042039901
178/178 [==============================] - 93s 521ms/step - loss: 0.1165 - accuracy: 0.9647 - val_loss: 0.1532 - val_accuracy: 0.9410
Epoch 3/3
178/178 [==============================] - ETA: 0s - loss: 0.0647 - accuracy: 0.9795
Epoch: 2. Reducing Learning Rate from 0.000640000042039901 to 0.0005120000569149852
178/178 [==============================] - 97s 544ms/step - loss: 0.0647 - accuracy: 0.9795 - val_loss: 0.0628 - val_accuracy: 0.9795


In [10]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print("Accuracy = " + str(np.round(test_acc * 100, 3)) + " %")

40/40 - 3s - loss: 0.0628 - accuracy: 0.9795 - 3s/epoch - 84ms/step
Accuracy = 97.954 %


In [11]:
filename = '../project/prediction_models/bt_bins_2.bin'
pk.dump(model, open(filename, 'wb'))

INFO:tensorflow:Assets written to: ram://f30cba7c-d609-4155-aeb8-852ac7ad0510/assets


In [ ]:
model = pk.load(open(filename, 'rb'))